<a href="https://colab.research.google.com/github/mracy/Chatbot-using-Google-colab-/blob/main/PalmMindTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install necessary pip

In [ ]:
# Install necessary packages in Google Colab
!pip install nltk transformers datefinder

In [ ]:
import nltk
from transformers import pipeline
import re
import datefinder
from datetime import datetime, timedelta

# Ensure necessary resources are available
nltk.download('punkt')

# Load the model for text generation (using DialoGPT-medium)
chatbot = pipeline("text-generation", model="microsoft/DialoGPT-medium")

# Static document (haircut appointment example with a special service)
haircut_info = """
Welcome to The Hair Stylist Salon! We are excited to offer a range of haircut and styling services.
Our services include:
1. Men's Haircut – Clean and Classic Styles
2. Women's Haircut – Modern and Trendy Cuts
3. Children's Haircut – Fun and Comfortable Styles
4. Beard Grooming – Precision Trimming and Shaping
5. Special Service of the Day: Keratin Treatment for Smooth and Shiny Hair

We also offer a variety of hair care products to keep your hair looking great between visits.
Our salon is located at:
The Hair Stylist Salon
123 Beauty Road, Kathmandu, Nepal
We are open every day from 9:00 AM to 7:00 PM.
For appointments, please ask for an appointment, and our team will be happy to assist you!
"""

# Enhanced Function to answer user queries from the salon info, including special services
def answer_query(query):
    # Define a list of service-related keywords/phrases
    service_keywords = ["services", "today's special", "special of the day", "haircut options", "special service"]

    # Define a list of discount-related keywords
    discount_keywords = ["discount", "sale", "offer", "promotion", "special offer", "5% off", "10% off"]

    # Check if the query contains any service-related keywords
    if any(keyword in query.lower() for keyword in service_keywords):
        # Check if it's specifically asking about the special service of the day
        if "today's special" in query.lower() or "special service" in query.lower() or "special" in query.lower():
            return "Special Service of the Day: Keratin Treatment for Smooth and Shiny Hair!"
        return haircut_info

    # Check if the query mentions a discount or offer
    if any(keyword in query.lower() for keyword in discount_keywords):
        return "We are currently offering discounts ranging from 5% to 10% on selected services! Be sure to ask our staff about today's special offers."

    # Check for location-related queries
    location_keywords = ["location", "where are you", "where is the salon", "address", "where"]
    if any(keyword in query.lower() for keyword in location_keywords):
        return "Our salon is located at: 123 Beauty Road, Kathmandu, Nepal."

    # Check for haircut-related queries (e.g., "I want a hair cut", "My daughter wants a haircut")
    haircut_keywords = ["hair", "haircut", "hairstyle", "daughter", "son", "child"]
    if any(keyword in query.lower() for keyword in haircut_keywords):
        return "It sounds like you are interested in a haircut! Please provide your details to book an appointment and our team will be happy to assist you."

    # Otherwise, use the chatbot to generate a response
    input_text = f"User: {query}\nBot:"
    response = chatbot(input_text, max_length=50, num_return_sequences=1)
    return response[0]['generated_text'].replace(input_text, "")

# Function to extract and handle date from user query
def extract_date(query):
    today = datetime.today()
    day_of_week = today.weekday()  # Monday = 0, Sunday = 6

    # Handle specific days of the week like "next Monday", "next Friday"
    days_map = {
        "next monday": 0,
        "next tuesday": 1,
        "next wednesday": 2,
        "next thursday": 3,
        "next friday": 4,
        "next saturday": 5,
        "next sunday": 6,
    }

    # Check for relative weekday-based queries like "next Monday"
    for key, value in days_map.items():
        if key in query.lower():
            days_to_add = (value - day_of_week + 7) % 7
            return (today + timedelta(days=days_to_add)).strftime("%Y-%m-%d")

    # Handle more general date extraction using datefinder
    matches = list(datefinder.find_dates(query))
    if matches:
        return matches[0].strftime("%Y-%m-%d")
    else:
        return None

# Validate email
def validate_email(email):
    regex = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    if re.match(regex, email):
        return True
    return False

# Validate phone number (simple validation for 10-digit number)
def validate_phone(phone):
    if re.match(r'^\d{10}$', phone):
        return True
    return False

# Function to collect user information
def collect_user_info():
    user_info = {}
    print("Please provide your details to book an appointment.")
    user_info['Name'] = input("Enter your name: ")

    # Validate phone number
    while True:
        user_info['Phone'] = input("Enter your phone number (10 digits): ")
        if not validate_phone(user_info['Phone']):
            print("Invalid phone number. Please enter a valid 10-digit phone number.")
        else:
            break

    # Validate email
    while True:
        user_info['Email'] = input("Enter your email address: ")
        if not validate_email(user_info['Email']):
            print("Invalid email format. Please enter a valid email address.")
        else:
            break

    return user_info

# Function to simulate appointment booking
def book_appointment(user_info, date):
    print(f"Congratulations {user_info['Name']}, you have successfully booked an appointment!")
    print(f"Appointment Details:\nName: {user_info['Name']}\nPhone: {user_info['Phone']}\nEmail: {user_info['Email']}\nDate: {date}")
    return f"Appointment for {user_info['Name']} booked on {date}."

# Conversational loop
def chatbot_conversation():
    print("Hello! Welcome to our Hair Stylist Salon Bot. How can I assist you today?")

    while True:
        query = input("You: ")

        # Check if the user wants to book an appointment
        if "call me" in query.lower() or "book an appointment" in query.lower():
            user_info = collect_user_info()
            date_query = input("Please mention the preferred date for your appointment (e.g., next Monday): ")
            appointment_date = extract_date(date_query)

            if appointment_date:
                booking_confirmation = book_appointment(user_info, appointment_date)
                print(booking_confirmation)
            else:
                print("Sorry, I couldn't extract a valid date from your input.")

        # Check if the user wants to exit the chatbot
        elif "exit" in query.lower():
            print("Bot: Goodbye! Have a great day!")
            break

        # Otherwise, answer the query based on the salon info
        else:
            response = answer_query(query)
            print("Bot: " + response)

# Run the chatbot
chatbot_conversation()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Hello! Welcome to our Hair Stylist Salon Bot. How can I assist you today?
You: Hello


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Bot:  Hello
You: Where is your shop?
Bot: Our salon is located at: 123 Beauty Road, Kathmandu, Nepal.
You: What are you services?
Bot: 
Welcome to The Hair Stylist Salon! We are excited to offer a range of haircut and styling services.
Our services include:
1. Men's Haircut – Clean and Classic Styles
2. Women's Haircut – Modern and Trendy Cuts
3. Children's Haircut – Fun and Comfortable Styles
4. Beard Grooming – Precision Trimming and Shaping
5. Special Service of the Day: Keratin Treatment for Smooth and Shiny Hair

We also offer a variety of hair care products to keep your hair looking great between visits.
Our salon is located at:
The Hair Stylist Salon
123 Beauty Road, Kathmandu, Nepal
We are open every day from 9:00 AM to 7:00 PM.
For appointments, please ask for an appointment, and our team will be happy to assist you!

You: Do you offer discount?
Bot: We are currently offering discounts ranging from 5% to 10% on selected services! Be sure to ask our staff about today's special 